In [27]:
# import dependencies 
import tensorflow as tf
import numpy as np 
import pandas as pd 
from datetime import datetime, date, time
import os 
import glob
import csv

##importing dependencies 
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS 

token = "A1hvR7pmPINW66Wuxbb7dpwR74TukBJIC7IuP3ADhr4xTx0Zzx-JZxzgInUUFbikMsg_bNo2TI4QnbXnUIOkDg=="
org = "ThirdYearProject"
bucket = "86cf3b0e0c4d6c44"

##connect to influxdb
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org,timeout=300000)

##DATA INTO INFLUXDB
##creating object of write API 
##synchronous specifies that there will be multiple rows of data being stored
##at a time
write_api = client.write_api(write_options=SYNCHRONOUS,batch_size=10000,flush_inveral=5000)

file_list=pd.read_csv(r"C:\Users\adela\OneDrive - University of Bristol\Documents\Third Year Project\Data\dataInDB.csv")
col_names=['HEADER']

data = pd.read_csv(r"C:\Users\adela\OneDrive - University of Bristol\Documents\Third Year Project\allCSV\channel_1003_segment_28.csv",index_col=None,names=col_names)

data[['DateTime','Q-Factor', 'Transmit Power (dBm)', 'Chromatic Dispersion(ps/nm)', 'Polarization Mode Dispersion']]=data['HEADER'].apply(lambda x: pd.Series(str(x).split("\t")))
data.drop('HEADER', axis=1,inplace=True)
convert_dict={'Q-Factor':float,
             'Transmit Power (dBm)':float,
             'Chromatic Dispersion(ps/nm)':float
             }
data['Polarization Mode Dispersion'] = pd.to_numeric(data['Polarization Mode Dispersion'], downcast="float",errors='coerce')
data.fillna(0)
data=data.astype(convert_dict)
data['Polarization Mode Dispersion']=data['Polarization Mode Dispersion'].astype('float64')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36160 entries, 0 to 36159
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   DateTime                      36160 non-null  object 
 1   Q-Factor                      36160 non-null  float64
 2   Transmit Power (dBm)          36160 non-null  float64
 3   Chromatic Dispersion(ps/nm)   36160 non-null  float64
 4   Polarization Mode Dispersion  36160 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.4+ MB


In [28]:
#response = write_api.write(bucket, record=data, data_frame_measurement_name='Network Prediction')

In [32]:
list = file_list.values.tolist()
len(list)

938

In [38]:
#if (['channel_1003_segment_28.csv']) in list:
    #print('Exists.')
#list.append(['channel_1003_segment_28.csv'])
#if (['channel_1003_segment_28.csv']) in list:
    print('Exists.')
#len(list)
#list.remove(['channel_1003_segment_28.csv'])
#print(list)

IndentationError: unexpected indent (3285502522.py, line 5)

In [40]:
col_names=['HEADER']
csv_directory=r"C:\Users\adela\OneDrive - University of Bristol\Documents\Third Year Project\allCSV"
count=len(list)
for filename in os.listdir(csv_directory):
    if ([filename]) not in list:
        file_list.append([filename])
        count+=1
        data = pd.read_csv(csv_directory+'/'+filename,index_col=None,names=col_names)

        #giving column titles to data
        #data.columns = ['DateTime','Q-Factor', 'Transmit Power (dBm)', 'Chromatic Dispersion(ps/nm)', 'Polarization Mode Dispersion']
        data[['DateTime','Q-Factor', 'Transmit Power (dBm)', 'Chromatic Dispersion(ps/nm)', 'Polarization Mode Dispersion']]=data['HEADER'].apply(lambda x: pd.Series(str(x).split("\t")))
        data.drop('HEADER', axis=1,inplace=True)
        data['Polarization Mode Dispersion'] = data['Polarization Mode Dispersion'].replace('?',0)
        convert_dict={'Q-Factor':float,
                     'Transmit Power (dBm)':float,
                     ' ':float,
                      'Polarization Mode Dispersion':float
                     }
        data=data.astype(convert_dict)
        #clearning up data -> replacing ? with 0 (maybe should replace it with the average)
        #data['Polarization Mode Dispersion'] = pd.to_numeric(data['Polarization Mode Dispersion'], downcast="float",errors='coerce')
        #data.fillna(0)
        data['DateTime'] = pd.to_datetime(data['DateTime'], format="%Y.%m.%d.%H.%M.%S")
        data['DateTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
        data.set_index(data['DateTime'], drop = True, inplace = True)

        #data.drop('DateTime', axis=1, inplace=True)
        data.drop('DateTime', axis=1, inplace=True)
    
        print(count)

        ##writing data into influxdb
        response = write_api.write(bucket, record=data, data_frame_measurement_name='Network Prediction')

C:\Users\adela\AppData\Local\Temp\ipykernel_14764\827550484.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_list.append([filename])


942


C:\Users\adela\AppData\Local\Temp\ipykernel_14764\827550484.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_list.append([filename])


943


C:\Users\adela\AppData\Local\Temp\ipykernel_14764\827550484.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_list.append([filename])


...

3366


C:\Users\adela\AppData\Local\Temp\ipykernel_14764\827550484.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_list.append([filename])


3367


C:\Users\adela\AppData\Local\Temp\ipykernel_14764\827550484.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file_list.append([filename])


KeyboardInterrupt: 

In [49]:
print(count)

3368


In [50]:
print(file_list)

     channel_1000_segment_28.csv
0    channel_1001_segment_28.csv
1    channel_1002_segment_28.csv
2    channel_1003_segment_28.csv
3    channel_1004_segment_28.csv
4    channel_1005_segment_28.csv
..                           ...
933  channel_1842_segment_51.csv
934  channel_1000_segment_28.csv
935  channel_1001_segment_28.csv
936  channel_1002_segment_28.csv
937  channel_1003_segment_28.csv

[938 rows x 1 columns]


In [46]:
import csv

In [47]:
rows = [[item] for item in file_list]
print(rows)
file = open(r"C:\Users\adela\OneDrive - University of Bristol\Documents\Third Year Project\Data\dataInDB.csv", 'a+',newline ='')
with file:
    write = csv.writer(file)
    write.writerows(rows)

[['channel_1000_segment_28.csv']]
